* Base de datos: 2007-2020
* Tipo corrupción: Intensa

## 1. Cargar datos

In [1]:
import warnings
warnings.filterwarnings( 'ignore' )

In [2]:
import pandas as pd
import numpy as np
import pickle
import joblib
from glob import glob

In [3]:
path      = r'..\..\..\input\prediction_data\ejecucion_7\b0_ci_siaf\preprocessed_data\*.csv'
path_list = glob( path )
dfs       = []

for i, base in enumerate( path_list ):
    df = pd.read_csv( base, dtype = { 'ubigeo': str, 'year': str } )
    dfs.append( df )

## 2. Hacer predicciones

In [4]:
optimal_model = joblib.load( r'..\..\..\output\ejecucion_6\models\base0\model_ci_Random Forest Classifier_2.joblib' )

In [5]:
for i, df in enumerate( dfs ):
    
    ubigeo_column                        = dfs[ i ][ 'ubigeo' ].copy()
    year_column                          = dfs[ i ][ 'year' ].copy()
    dfs[ i ]                             = dfs[ i ].drop( [ 'ubigeo', 'year' ], axis = 1 )
    
    y_pred_class_o                       = optimal_model.predict( dfs[ i ] )
    y_pred_prob_o                        = optimal_model.predict_proba( dfs[ i ] )[ :, 1 ]  
    dfs[ i ][ 'prob_ci' ]                = y_pred_prob_o
    dfs[ i ][ 'class_ci_default' ]       = y_pred_class_o
    dfs[ i ][ 'class_ci_calculated_50' ] = dfs[ i ][ 'prob_ci' ].apply( lambda x: 1 if x > 0.5 else 0 )
    dfs[ i ][ 'class_ci_calculated_60' ] = dfs[ i ][ 'prob_ci' ].apply( lambda x: 1 if x > 0.6 else 0 )
    dfs[ i ][ 'class_ci_calculated_70' ] = dfs[ i ][ 'prob_ci' ].apply( lambda x: 1 if x > 0.7 else 0 )
    dfs[ i ][ 'class_ci_100' ]           = ( dfs[ i ][ 'prob_ci' ].rank( ascending = False, method = 'first' ) <= 100 ).astype( int )
    
    dfs[ i ][ 'ubigeo' ]                = ubigeo_column
    dfs[ i ][ 'year' ]                  = year_column
    
    dfs[ i ]                            = dfs[ i ][ [ 'ubigeo', 'year', 'prob_ci', 'class_ci_default', 
                                                      'class_ci_calculated_50', 'class_ci_calculated_60',
                                                      'class_ci_calculated_70', 'class_ci_100' ] ]

In [6]:
complete_base = pd.concat( dfs, axis = 0 )

In [7]:
complete_base

,ubigeo,year,prob_ci,class_ci_default,class_ci_calculated_50,class_ci_calculated_60,class_ci_calculated_70,class_ci_100
0,010101,2007.0,0.836000,1.0,1,1,1,0
1,010102,2007.0,0.848000,1.0,1,1,1,1
2,010103,2007.0,0.848000,1.0,1,1,1,1
3,010104,2007.0,0.848000,1.0,1,1,1,1
4,010105,2007.0,0.848000,1.0,1,1,1,1
...,...,...,...,...,...,...,...,...
1869,250302,2020.0,0.412000,0.0,0,0,0,0
1870,250303,2020.0,0.700000,1.0,1,1,0,0
1871,250304,2020.0,0.767983,1.0,1,1,1,1
1872,250305,2020.0,0.691983,1.0,1,1,0,0


## 3. Exportar datos

In [8]:
complete_base.to_csv( r'..\..\..\input\prediction_data\ejecucion_7\b0_ci_siaf\predicted_data\base0_predicted_ci.csv', index = False )